In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
model = IALSRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])
earlystopping_kwargs =  {"validation_every_n": 10,
                              "stop_on_validation": True,
                              "evaluator_object": evaluator_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": "MAP",
                              }

EvaluatorHoldout: Ignoring 2600 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [5]:
import optuna as op
def objective(trial):
    num_factors = trial.suggest_int("num_factors", 1, 200)
    epochs = 300
    confidence_scaling = trial.suggest_categorical("confidence_scaling", ["linear", "log"])
    alpha = trial.suggest_loguniform("alpha", 1e-4, 50.0)
    epsilon = trial.suggest_loguniform("epsilon", 1e-5, 10.0)
    reg = trial.suggest_loguniform("reg", 1e-5, 1.0)
    
    recommender = model(URM_train)
    recommender.fit(**trial.params, **earlystopping_kwargs)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
    "num_factors": 91,
    "epochs": 50,
    "confidence_scaling": "log",
    "alpha": 8.931839513881036,
    "epsilon": 6.099966408152877,
    "reg": 0.0003668646666587094,
}

# {"num_factors": 91, "epochs": 50, "confidence_scaling": "log", "alpha": 8.931839513881036, "epsilon": 6.099966408152877, "reg": 0.0003668646666587094}
study_name = "IALS-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
#op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

[I 2023-12-21 03:39:01,364] Using an existing study with name 'IALS-study' instead of creating a new one.
/tmp/ipykernel_2740/1158101635.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform("alpha", 1e-4, 50.0)
/tmp/ipykernel_2740/1158101635.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  epsilon = trial.suggest_loguniform("epsilon", 1e-5, 10.0)
/tmp/ipykernel_2740/1158101635.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  reg = trial.suggest_loguniform("reg", 1e-5, 1.0)


IALSRecommender: URM Detected 448 ( 3.5%) users with no interactions.
IALSRecommender: URM Detected 320 ( 1.4%) items with no interactions.
IALSRecommender: Epoch 1 of 300. Elapsed time 7.26 sec
IALSRecommender: Epoch 2 of 300. Elapsed time 14.93 sec
IALSRecommender: Epoch 3 of 300. Elapsed time 22.17 sec
IALSRecommender: Epoch 4 of 300. Elapsed time 29.14 sec
IALSRecommender: Epoch 5 of 300. Elapsed time 36.11 sec
IALSRecommender: Epoch 6 of 300. Elapsed time 43.26 sec
IALSRecommender: Epoch 7 of 300. Elapsed time 50.43 sec
IALSRecommender: Epoch 8 of 300. Elapsed time 57.50 sec
IALSRecommender: Epoch 9 of 300. Elapsed time 1.08 min
IALSRecommender: Validation begins...
EvaluatorHoldout: Processed 10038 (100.0%) in 18.35 sec. Users per second: 547
IALSRecommender: epoch
10    0.019345
Name: MAP, dtype: object
IALSRecommender: New best model found! Updating.
IALSRecommender: Epoch 10 of 300. Elapsed time 1.50 min
IALSRecommender: Epoch 11 of 300. Elapsed time 1.61 min
IALSRecommender: 

[I 2023-12-21 03:59:28,674] Trial 21 finished with value: 0.019797309904901056 and parameters: {'num_factors': 59, 'confidence_scaling': 'linear', 'alpha': 0.056242207604168544, 'epsilon': 0.04505171627503675, 'reg': 0.00038036792243579897}. Best is trial 13 with value: 0.020532302646767732.


IALSRecommender: URM Detected 448 ( 3.5%) users with no interactions.
IALSRecommender: URM Detected 320 ( 1.4%) items with no interactions.


/tmp/ipykernel_2740/1158101635.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform("alpha", 1e-4, 50.0)
/tmp/ipykernel_2740/1158101635.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  epsilon = trial.suggest_loguniform("epsilon", 1e-5, 10.0)
/tmp/ipykernel_2740/1158101635.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  reg = trial.suggest_loguniform("reg", 1e-5, 1.0)


...

IALSRecommender: Epoch 1 of 300. Elapsed time 2.90 min
IALSRecommender: Epoch 2 of 300. Elapsed time 5.80 min
IALSRecommender: Epoch 3 of 300. Elapsed time 8.69 min
IALSRecommender: Epoch 4 of 300. Elapsed time 11.59 min
IALSRecommender: Epoch 5 of 300. Elapsed time 14.49 min
IALSRecommender: Epoch 6 of 300. Elapsed time 17.39 min
IALSRecommender: Epoch 7 of 300. Elapsed time 20.28 min
IALSRecommender: Epoch 8 of 300. Elapsed time 23.18 min
IALSRecommender: Epoch 9 of 300. Elapsed time 26.08 min
IALSRecommender: Validation begins...
EvaluatorHoldout: Processed 10038 (100.0%) in 2.83 min. Users per second: 59
IALSRecommender: epoch
10    0.022408
Name: MAP, dtype: object
IALSRecommender: New best model found! Updating.
IALSRecommender: Epoch 10 of 300. Elapsed time 31.81 min
IALSRecommender: Epoch 11 of 300. Elapsed time 34.70 min
IALSRecommender: Epoch 12 of 300. Elapsed time 37.60 min
IALSRecommender: Epoch 13 of 300. Elapsed time 40.50 min
IALSRecommender: Epoch 14 of 300. Elapsed ti

[I 2023-12-25 21:57:34,788] Trial 72 finished with value: 0.022750520245290135 and parameters: {'num_factors': 200, 'confidence_scaling': 'log', 'alpha': 0.782018989391869, 'epsilon': 0.022419573867596614, 'reg': 0.07115760266087914}. Best is trial 69 with value: 0.022855099036360273.


IALSRecommender: URM Detected 448 ( 3.5%) users with no interactions.
IALSRecommender: URM Detected 320 ( 1.4%) items with no interactions.


/tmp/ipykernel_2740/1158101635.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform("alpha", 1e-4, 50.0)
/tmp/ipykernel_2740/1158101635.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  epsilon = trial.suggest_loguniform("epsilon", 1e-5, 10.0)
/tmp/ipykernel_2740/1158101635.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  reg = trial.suggest_loguniform("reg", 1e-5, 1.0)


IALSRecommender: Epoch 1 of 300. Elapsed time 2.90 min
IALSRecommender: Epoch 2 of 300. Elapsed time 5.80 min
IALSRecommender: Epoch 3 of 300. Elapsed time 8.70 min
IALSRecommender: Epoch 4 of 300. Elapsed time 11.59 min
IALSRecommender: Epoch 5 of 300. Elapsed time 14.49 min
IALSRecommender: Epoch 6 of 300. Elapsed time 17.39 min
IALSRecommender: Epoch 7 of 300. Elapsed time 20.28 min
IALSRecommender: Epoch 8 of 300. Elapsed time 23.17 min
IALSRecommender: Epoch 9 of 300. Elapsed time 26.07 min
IALSRecommender: Validation begins...
EvaluatorHoldout: Processed 10038 (100.0%) in 1.84 min. Users per second: 91
IALSRecommender: epoch
10    0.021734
Name: MAP, dtype: object
IALSRecommender: New best model found! Updating.
IALSRecommender: Epoch 10 of 300. Elapsed time 30.81 min
IALSRecommender: Epoch 11 of 300. Elapsed time 33.71 min
IALSRecommender: Epoch 12 of 300. Elapsed time 36.60 min
IALSRecommender: Epoch 13 of 300. Elapsed time 39.48 min
IALSRecommender: Epoch 14 of 300. Elapsed ti

In [9]:
study.best_params

{'topK': 101, 'alpha': 0.3026342852596128, 'beta': 0.058468783118329024}

In [10]:
final = model(URM_train_validation)
final.fit(**study.best_params)

RP3betaRecommender: URM Detected 245 ( 1.9%) users with no interactions.
RP3betaRecommender: URM Detected 110 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4027.41 column/sec. Elapsed time 5.52 sec


In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2100 (16.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10538 (100.0%) in 5.18 sec. Users per second: 2033


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.090378                 0.157221  0.133194  0.045665    0.078973   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.259414  0.143182  0.107686  0.509869      0.339649  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.833834          0.425146    0.833834       0.057877   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.145296                   0.997617             0.165954   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   